# exemplo_nb_base

Aqui é recomendado adicionar um comentário explicando o contexto do notebook, tabelas e colunas que são adicionadas, e uma descrição sobre a tabela de saída.


## Configurações Iniciais

### Preparação do Ambiente PySpark

#### Instalando as Dependências

In [1]:
# Instala o Java 8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Baixa o Apache Spark 3.4.1 com Hadoop 3 (versão estável e recente)
!wget -q https://archive.apache.org/dist/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz

# Descompacta o arquivo baixado
!tar xf spark-3.4.1-bin-hadoop3.tgz

# Instala a biblioteca findspark
!pip install -q findspark

#### Editando as Variáveis de Ambiente

In [2]:
import os

# Define a variável de ambiente do Java
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

# Define a variável de ambiente do Spark
os.environ["SPARK_HOME"] = "/content/spark-3.4.1-bin-hadoop3"

from pyspark.sql import SparkSession

# Cria a SparkSession
spark = SparkSession.builder.master("local[*]").appName("E-commerce Dataset").getOrCreate()

### Importando as Bibliotecas

In [3]:
from pyspark.sql import functions as F

### Carregando os Dados

In [ ]:
"""
Carrega o dataset de clientes do Olist. Esse é um exemplo de importação de dados 
de um arquivo CSV, porém, normalmente é utilizado o comando spark.read.table,
para ingerir tabelas de um data lake ou data warehouse.

Não é recomendado fazer grandes transformações nessa célula,
pois o objetivo é apenas carregar o dataset, 
fazendo o select das colunas que serão utilizadas ou o drop de colunas inúteis.
""" 
df_customers_dataset = (
    spark.read.csv(
        "/content/olist_customers_dataset.csv",
        header=True,
        inferSchema=True
    )
    .drop("customer_city", "customer_state")
)

## Transformação dos Dados

A partir dessa célula é possível fazer diversas transformações e cálculos em cima das tabelas importadas. Normalmente são feitos joins entre tabelas e a aplicação de alguns cálculos com base nas colunas que foram adicionadas.

## Salvamento dos Dados

Aqui é feita a modelagem dos dados, onde é definido o schema final da tabela que será escrita, criação de relações entre as demais tabelas e adição de descrição de colunas e tabelas.

### Observação Importante:
Nessa oficina não temos um data lake para armazenar os dados, então essa etapa da modelagem deve ser ignorada.

In [ ]:
%sql
CREATE TABLE IF NOT EXISTS TABLE dim_clientes (
  customer_id STRING NOT NULL COMMENT "ID único do cliente, associado a um pedido específico. Chave primária.",
  customer_zip_code_prefix STRING COMMENT "CEP de cadastro do cliente.",
  client_name STRING COMMENT "Nome do cliente.",
  CONSTRAINT fk_dim_clientes_geo FOREIGN KEY(customer_zip_code_prefix) REFERENCES dim_geolocalizacao(geolocation_zip_code_prefix)
)
COMMENT "Tabela de dimensão de clientes do Olist, contendo informações básicas dos clientes."
CLUSTER BY (customer_id);


Por fim, a escrita da tabela final

In [ ]:
# Exemplo de escrita da tabela final em um data lake
(
    df_customers_dataset
    .write.mode("overwrite")
    .format("delta")
    .saveAsTable("ofc_dev_master.clients.dim_clientes")
)

# Exemplo de um arquivo CSV, para o caso da oficina
(
    df_customers_dataset
    .coalesce(1)
    .write
    .option("header", "true")
    .mode("overwrite")
    .csv("/transformed_data/dim_clientes")
)

Célula utilizada para o download do CSV gerado:

In [ ]:
import glob
from google.colab import files
output_dir = "/transformed_data/dim_clientes"

try:
    # Encontra o arquivo CSV específico criado pelo Spark
    csv_file_path = glob.glob(f"{output_dir}/part-*.csv")[0]
    print(f"Arquivo encontrado para download: {csv_file_path}")

    # Usa a biblioteca do Colab para iniciar o download
    print("Iniciando o download...")
    files.download(csv_file_path)
    print("Download concluído!")

except IndexError:
    print(f"Erro: Nenhum arquivo CSV foi encontrado no diretório '{output_dir}'.")
    print("Verifique se o passo de escrita do Spark foi executado com sucesso.")